In [ ]:
import numpy as np
import random
import sympy as sp
import numpy as np
import numpy.linalg as la

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
w = sp.symbols('w_1:3')
s = sp.symbols('s_1:3')

def getBlockA(w,s):
    A = None
    for v,r in zip(w,s):
        X = sp.Matrix([[r, v],[-v,r]])
        if A is None:
            A = X
        else:
            A = sp.BlockDiagMatrix(A, X)
    return A
sp.pprint(getBlockA(w,s))

def getA(w,s):
    A = sp.zeros(len(2*w), len(2*w))
    for i,(v,r) in enumerate(zip(w,s)):
        XX = sp.Matrix([[r, v],[-v,r]])
        ix = 2*i
        A[ix, ix] = r
        A[ix+1, ix+1] = r
        A[ix, ix+1] = v
        A[ix+1, ix] = -v
    return A

sp.pprint(getA(w,s))

In [ ]:
import sympy as sp
import numpy as np
import numpy.linalg as la

w = sp.symbols('w_1')
s = sp.symbols('s_1')
b = sp.symbols('b_1:3')
d = sp.symbols('d_1')
x = sp.symbols('x_1:3')
c = sp.symbols('c_1:3')

u = sp.symbols('u_1')

X = sp.Matrix([x]).T
U = sp.Matrix([u])


A = sp.Matrix([[s, w],[-w,s]])
B = sp.Matrix([b]).T
C = sp.Matrix([c])
D = sp.Matrix([d])

AA = sp.lambdify((s, w), A)
BB = sp.lambdify(b, B)
CC = sp.lambdify(c, C)
DD = sp.lambdify(d, D)

z = X.T.row_join(sp.Matrix([s])).row_join(sp.Matrix([w])).row_join(B.T).row_join(C).row_join(D).T
#print z

fx = A*X+B*U
f = sp.Matrix([0 for _ in range(len(z)-len(x))])
f = fx.col_join(f)

print "f="; sp.pprint(f)
ff = sp.lambdify(z.T.row_join(U), f)

print "F="; sp.pprint(f.jacobian(z))#; print len(f.jacobian(z))
FF = sp.lambdify(z.T.row_join(U), f.jacobian(z))

h = C*X+D*U
print "h="; sp.pprint(h)
hh = sp.lambdify(z.T.row_join(U), h)

print "H="; sp.pprint(h.jacobian(z))#; print len(h.jacobian(z))
HH = sp.lambdify(z.T.row_join(U), h.jacobian(z))

#sp.pprint((h.jacobian(z)*h.jacobian(z).T))

In [ ]:
def _c(M):
    return M.tolist()[0]

zu_test = np.matrix([0, 0, -1, 1, -2, 1, 0.1, 0.2, 0, 2])

#print zu_test
#print ff(*(zu_test.tolist()[0]))

print ff(*_c(zu_test))

In [ ]:
rho = 0.1

#R = np.eye(len(z))
R = 1
#print np.diag([sym for sym in z])
print R

q = np.array([])
for sym in z:
    if sym in X.T.row_join(C):
        q = np.hstack((q, 0))
    else:
        q = np.hstack((q, 1))

Q = rho * np.matrix(np.diag(q))
print Q

In [ ]:
import control as con
#ss = con.matlab.ss(A_k, b_k.T, c_k, d_k)
#T, yout = con.matlab.step(ss)
#plt.plot(yout, T)

Af = np.matrix([[  -2, 4],
                [  -4, -2]])

Bf = np.matrix([[ 1],
                [ 1]])

Cf = np.matrix([[ 1/0.3, 0.]])

Df = np.matrix([[ 0.]])

ss2 = con.matlab.ss(Af, Bf, Cf, Df)
T2, yout2 = con.matlab.step(ss2)
plt.plot(yout2, T2)
plt.show()
print con.dcgain(ss2)
print ss2.pole()

In [ ]:
### Testsystem ####
def calc_TestSystem(x, u):
    global Af, Bf, Cf, Df

    #Af = np.matrix([[  -10, 20],
    #                [  -20, -10]])

    #Bf = np.matrix([[ 1],
    #                [ 1]])

    #Cf = np.matrix([[ 1/0.06, 0.]])

    #Df = np.matrix([[ 0.]])
    
    #print Af
    #print x
    #print "res"
    #print Af.dot(x)
    #print Bf.dot(u)
    #print "res end"
    x_dot = Af.dot(x) + Bf.dot(u)
    y = Cf.dot(x) + Df.dot(u)
    
    #print x_dot
    return y, x_dot

In [ ]:
# return real part of a vector
def real_vector(vector):
    return map(lambda x: x.real, vector)

# return imaginary part of a vector
def imag_vector(vector):
    return map(lambda x: x.imag, vector)

In [ ]:
random.seed(42) # Set the random number generator to a fixed sequence.
# initial
# A
s_k0 = np.matrix([-2]) # real Teil
w_k0 = np.matrix([2])  # (+/-) imag Teil

# b
b_k0 = np.matrix([-2, 1])

# c
c_k0 = np.matrix([0.1, 0.2])

# d
d_k0 = np.matrix([0])


P_k0 = np.eye(len(z))

x_k0 = np.matrix([0, 0]).T
#print x_k0
u_k0 = np.matrix([0.1])

T = 0.001


# First values
x_k = x_k0
x_t = x_k0
u_k = u_k0
z_k = np.hstack((x_k0.T, s_k0, w_k0, b_k0, c_k0, d_k0)).T
#print "z_k:", z_k

P_k = P_k0
poles = []
y = []
y_e = []

u = []

# Loop start
for c, t in enumerate(np.arange(T, 30, T)):

    #from IPython.core.debugger import Tracer; Tracer()() 
    
    # Every two seconds new step
    if np.isclose(t%2, 0, T):
        u_k = u_k0 + random.uniform(-1, 1)
        print "step:", t, "u:", u_k
        poles.append(np.array([complex(s_k, w_k), complex(s_k,-w_k)]))
        #print "s_k:\n", s_k
        #print "w_k:\n", w_k
        #print "b_k:\n", b_k
        #print "c_k:\n", c_k
        #print "d_k:\n", d_k
    
    #print "x_k:\n", x_k
    y_t, x_dot = calc_TestSystem(x_t, u_k)
    #print "y_k:\n", y_k
        
    y.append(y_t[0,0])
    u.append(u_k[0,0])
    
    ##### Reconstruction from z
    #print "z_k:", z_k    
    
    x_k = np.matrix(z_k.T[0, 0:2], dtype=np.float).T
    #print "x_k:\n",x_k
    s_k = np.matrix(z_k.T[0, 2], dtype=np.float)
    #print s_k
    w_k = np.matrix(z_k.T[0, 3], dtype=np.float)
    #print w_k
    b_k = np.matrix(z_k.T[0, 4:6], dtype=np.float)
    #print b_k
    c_k = np.matrix(z_k.T[0, 6:8], dtype=np.float)
    d_k = np.matrix(z_k.T[0, 8], dtype=np.float)

    ## System
    A_k = AA(s_k, w_k)
    B_k = BB(*_c(b_k))
    C_k = CC(*_c(c_k))
    D_k = DD(*_c(d_k))


    ##### Evaluation
    # State space
    dx_k = A_k.squeeze().dot(x_k) + B_k.dot(u_k)
    y_k = C_k.dot(x_k) + D_k.dot(u_k)
    y_e.append(y_k[0,0])

    # Concate values for lambdafied Jacobians
    zu_k = np.hstack((z_k.T, u_k))
    #print "zu_k:\n", zu_k
    
    h_k = np.matrix(hh(*_c(zu_k))) # h = y_k (predicted)
    #print "h_k:\n", h_k
    H_k = np.matrix(HH(*_c(zu_k))) # h.Jacobian
    #print "H_k:\n", H_k

    
    f_k = np.matrix(ff(*_c(zu_k))) # f
    #print "f_k:\n", f_k
    F_k = np.matrix(FF(*_c(zu_k))) #f.Jacobian
    #print "F_k:\n", F_k
    
    ##### Kalman Filter

    # Prediction

    #print "P_k:\n", P_k
    #print "H_k.T:\n", H_k.T
    
    K_k = P_k.dot(H_k.T).dot(la.inv(H_k.dot(P_k).dot(H_k.T) + R))
    #print "K_k:\n", K_k

    Ps_k = (np.eye(len(z)) - K_k.dot(H_k)).dot(P_k) # P*_k
    #print "Ps_k:\n", Ps_k

    # Correction
    P_k1 = Ps_k + T*(F_k.dot(Ps_k) + Ps_k.dot(F_k.T) + Q)
    #print "P_k1:\n", P_k1

    z_k1 = (z_k + T*f_k + K_k.dot((y_t - h_k)))
    #print "z_k1:", z_k1

    # State Propagation
    x_t = x_t + x_dot*T
    #print "x_k:\n",x_k
    
    z_k = np.matrix(z_k1)
    #print "z_k:\n", z_k
    
    P_k = np.matrix(P_k1).astype(np.float)
    #print "P_k:\n", P_k


# another approach to plot
real_part = real_vector(poles)
imag_part = imag_vector(poles)

# Display a window with a plot of real, imag
plt.plot(real_part, imag_part, 'b-')
plt.plot(real_vector(poles), imag_vector(poles), 'gx')
plt.plot(real_part[0], imag_part[0], 'b*')
plt.plot(real_part[-1], imag_part[-1], 'rx')

plt.show

In [ ]:
plt.plot(y, 'b')
plt.plot(u, 'r')
plt.plot(y_e, 'y-')

plt.show


In [ ]:
ss = con.matlab.ss(A_k, b_k.T, c_k, d_k)

#print A_k[:,:,0,0]
#print A_k.squeeze()
#print b_k
#print c_k
#print d_k

#print poles[-1]
print ss

In [ ]:
print con.dcgain(ss2)
print ss2.pole()
print con.dcgain(ss)
print ss.pole()

In [ ]:
T3, yout3 = con.matlab.step(ss2)
plt.plot(yout3, T3)
#plt.show()

T4, yout4 = con.matlab.step(ss)
plt.plot(yout4, T4)
plt.show()